<a href="https://colab.research.google.com/github/DmitryBal/Cup_Russia/blob/main/%D0%A1%D0%9C%D0%9E2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pickle
import math

from math import factorial

import numpy as np
import pandas as pd

In [ ]:
# V = 39
lam = 7.384
mu = 1.069
dTz = 0.141

m = 12
n = 7

# Система массового обслуживания (D|M|n)

In [ ]:
class Device:
    def __init__(self, device_id):
        self.device_id = device_id
        self.is_busy = False
        self.finish_time = None
        self.cur_request = None
        self.processing_time = None
        self.processed = 0
        self.was_busy = 0


    def finish(self, time):
      global requests
      if not self.is_busy: return
      print(f"DEVICE {self.device_id} HAS UNFINISHED JOB!")
      self.was_busy += time - requests[self.cur_request]["ts"]


    def new_request(self, request, write=True, override=False):
        global counter
        global actions
        global requests
        global state
        global left_time
        global next_query_deadline
        global min_finish
        self.is_busy = True
        self.processed += 1

        time = next_query_deadline if not override else override

        # Записать время и событие принятия заявки
        print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}')

        processing_time = np.random.exponential(1/mu)
        self.processing_time = processing_time
        if write:
          counter += 1
          actions.append([
            counter, #number
            time, # time recieved
            1, # type
            state, # state
            processing_time if processing_time < min_finish-time else min_finish-time, #min time left of work
            dTz, # next query in
            request, # request num
            self.device_id # device number
          ])

        self.finish_time = time + processing_time
        self.cur_request = request

        requests[request].update({
          "tq": time - requests[request]["tz"], # queue time
          "ts": time,
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
          "tk": self.device_id # who worked on request
        })

    def process_request(self):
        global counter
        global actions
        global devices
        global state
        global left_time
        global min_finish
        global next_query_deadline
        global min_device
        global devices
        global queue


        min_finish = np.inf
        min_device = None
        free_device = None

        for device in devices:
          if device.is_busy and device.finish_time < min_finish and device.device_id != self.device_id:
            min_finish = device.finish_time
            min_device = device
          elif not device.is_busy and free_device is None and device.device_id != self.device_id: free_device = device

        # Записать время и событие завершения обработки
        counter += 1
        state -= 1
        self.was_busy += self.processing_time

        print(f'{self.finish_time}: Устройство {self.device_id} обработало заявку {self.cur_request}, {counter}')

        requests[self.cur_request].update({
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
        })

        # requests[self.cur_request].update({
        # })

        # есть ли новые заявки в очереди, добавить новый реквест
        if len(queue) != 0 and counter <= 100:
            self.processed += 1

            actions.append([
              counter, #number
              self.finish_time, # time recieved
              2, # type
              state, # state
              self.processing_time if self.processing_time < min_finish-self.finish_time else min_finish-self.finish_time, #min time left of work
              next_query_deadline - self.finish_time, # next query in
              self.cur_request, # request num
              self.device_id # device number
            ])

            request = queue.pop(0)
            time = self.finish_time

            # Записать время и событие принятия заявки
            print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}')

            processing_time = np.random.exponential(1/mu)
            self.processing_time = processing_time

            self.finish_time = time + processing_time
            self.cur_request = request

            requests[request].update({
              "tq": time - requests[request]["tz"], # queue time
              "ts": time,
              "tw": self.processing_time, # for how long worked
              "te": self.finish_time, # when finished working
              "tk": self.device_id # who worked on request
            })
        elif counter <= 100:
          actions.append([
            counter, #number
            self.finish_time, # time recieved
            2, # type
            state, # state
            -1 if min_device is None else min_finish-self.finish_time,
            next_query_deadline - self.finish_time, # next query in
            self.cur_request, # request num
            self.device_id # device number
          ])

          self.finish_time = None
          self.cur_request = None
          self.is_busy = False

In [ ]:
counter = 0 # actions
request = 0 # queries
state = 0 # current state of query system
next_query_deadline = dTz # time when the new query appears
min_finish = np.inf
min_device = None

queue = [] # queries queue # .pop(0)
requests = {} # log of requests
actions = [] # log of actions

zl = []

devices = [Device(i+1) for i in range(n)]
# watcher
while counter < 100:
  min_finish = np.inf
  min_device = None
  free_device = None

  # get device that will end sooner than others
  for device in devices:
    if device.is_busy and device.finish_time < min_finish:
      min_finish = device.finish_time
      min_device = device
    elif not device.is_busy and free_device is None: free_device = device

  # is there any query that fires sooner?
  if next_query_deadline < min_finish:
    request += 1
    if len(zl): zl.append(zl[-1]+1)
    else: zl.append(1)

    if free_device is None:
      # all devices are busy. Add to queue
      counter += 1
      state += 1
      print(f'{next_query_deadline}: Новая заявка {request} встала в очередь, {counter}')
      actions.append([
            counter, #number
            next_query_deadline, # time recieved
            1, # type
            state, # state
            -1 if min_device is None else min_finish-next_query_deadline, #min time left of work
            dTz, # next query in
            request, # request num
            -1 # device number
        ])
      queue.append(request)

      requests[request] = {
          "j": request,
          "tz": next_query_deadline, # appeared
          "q": len(queue), # queue pos
          "tq": -1, # queue time
          "ts": -1, # start of working on request
          "tw": -1, # for how long worked
          "te": -1, # when finished working
          "tk": -1 # who worked on request
      }
    else:
      # At least one device is free
      state += 1
      requests[request] = {
          "j": request,
          "tz": next_query_deadline, # appeared
          "q": 0, # queue pos
          "tq": 0, # queue time
          "ts": next_query_deadline, # start of working on request
          "tw": 0, # for how long worked
          "te": -1, # when finished working
          "tk": free_device.device_id # who worked on request
      }
      free_device.new_request(request)
    next_query_deadline += dTz
  else:
    # Device processes query sooner
    zl.append(zl[-1]-1)
    min_device.process_request()


0.141: Устройство 1 приняло заявку 1, 0
0.282: Устройство 2 приняло заявку 2, 1
0.42299999999999993: Устройство 3 приняло заявку 3, 2
0.564: Устройство 4 приняло заявку 4, 3
0.6453722308105058: Устройство 2 обработало заявку 2, 5
0.705: Устройство 2 приняло заявку 5, 5
0.846: Устройство 5 приняло заявку 6, 6
0.9188247468683807: Устройство 2 обработало заявку 5, 8
0.94714143138899: Устройство 3 обработало заявку 3, 9
0.987: Устройство 2 приняло заявку 7, 9
1.128: Устройство 3 приняло заявку 8, 10
1.269: Устройство 6 приняло заявку 9, 11
1.41: Устройство 7 приняло заявку 10, 12
1.4205382600845: Устройство 1 обработало заявку 1, 14
1.551: Устройство 1 приняло заявку 11, 14
1.5722299035183094: Устройство 2 обработало заявку 7, 16
1.692: Устройство 2 приняло заявку 12, 16
1.756362813101491: Устройство 1 обработало заявку 11, 18
1.833: Устройство 1 приняло заявку 13, 18
1.974: Новая заявка 14 встала в очередь, 20
1.9982742945348089: Устройство 1 обработало заявку 13, 21
1.9982742945348089: У

In [ ]:
for device in devices:
  device.finish(actions[-1][1])

DEVICE 1 HAS UNFINISHED JOB!
DEVICE 2 HAS UNFINISHED JOB!
DEVICE 3 HAS UNFINISHED JOB!
DEVICE 4 HAS UNFINISHED JOB!
DEVICE 5 HAS UNFINISHED JOB!
DEVICE 6 HAS UNFINISHED JOB!
DEVICE 7 HAS UNFINISHED JOB!


In [ ]:
with open('actions1.pickle', 'wb') as handle:
    pickle.dump(actions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('requests1.pickle', 'wb') as handle:
    pickle.dump(requests, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('devices1.pickle', 'wb') as handle:
    pickle.dump(devices, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Вывод 1

In [ ]:
adf = pd.DataFrame(actions, columns=['ll', 't_соб', 'Type(l)', 'C(l)', 't_остмин', "t_ожз", "j(l)", "k(l)"])
adf['t_соб'] = adf['t_соб'].round(5)
adf['t_остмин'] = adf['t_остмин'].round(5)
adf['t_ожз'] = adf['t_ожз'].round(5)

adf['k(l)'] = adf['k(l)'].astype(int)
adf[:100]

,ll,t_соб,Type(l),C(l),t_остмин,t_ожз,j(l),k(l)
0,1,0.14100,1,1,1.27954,0.14100,1,1
1,2,0.28200,1,2,0.36337,0.14100,2,2
2,3,0.42300,1,3,0.22237,0.14100,3,3
3,4,0.56400,1,4,0.08137,0.14100,4,4
4,5,0.64537,2,3,0.30177,0.05963,2,2
...,...,...,...,...,...,...,...,...
95,96,7.61633,2,12,0.07113,0.13867,48,5
96,97,7.68746,2,11,0.03843,0.06754,30,6
97,98,7.72589,2,10,0.03690,0.02911,49,5
98,99,7.75500,1,11,0.00779,0.14100,55,-1


In [ ]:
# adf = adf.rename(columns=)

In [ ]:
df1, df2, df3, df4 = np.array_split(adf, 4)
tables = [df1, df2, df3, df4]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    latex = latex.replace("ll", "$\\mathbf{l}$")
    latex = latex.replace("t\_соб", "$\\mathbf{t_\\textbf{соб}}$")
    latex = latex.replace("Type(l)", "$\\mathbf{Type(l)}$")
    latex = latex.replace("C(l)", "$\\mathbf{C(l)}$")
    latex = latex.replace("t\_остмин", "$\\mathbf{t_\\textbf{остмин}}$")
    latex = latex.replace("t\_ожз", "$\\mathbf{t_\\textbf{ожз}}$")
    latex = latex.replace("j(l)", "$\\mathbf{j(l)}$")
    latex = latex.replace("k(l)", "$\\mathbf{k(l)}$")

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-10-a781772d1cc1>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print("\\begin{center} \\scalebox{0.7}{")
print(latex_tables[0])
print("}\\end{center}")

\begin{center} \scalebox{0.7}{
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 $\mathbf{l}$ &   $\mathbf{t_\textbf{соб}}$ &  $\mathbf{Type(l)}$ &  $\mathbf{C(l)}$ &  $\mathbf{t_\textbf{остмин}}$ &   $\mathbf{t_\textbf{ожз}}$ &  $\mathbf{j(l)}$ &  $\mathbf{k(l)}$ \\ \hline

  1 & 0.14100 &        1 &     1 &   1.27954 & 0.14100 &     1 &     1 \\ \hline
  2 & 0.28200 &        1 &     2 &   0.36337 & 0.14100 &     2 &     2 \\ \hline
  3 & 0.42300 &        1 &     3 &   0.22237 & 0.14100 &     3 &     3 \\ \hline
  4 & 0.56400 &        1 &     4 &   0.08137 & 0.14100 &     4 &     4 \\ \hline
  5 & 0.64537 &        2 &     3 &   0.30177 & 0.05963 &     2 &     2 \\ \hline
  6 & 0.70500 &        1 &     4 &   0.21382 & 0.14100 &     5 &     2 \\ \hline
  7 & 0.84600 &        1 &     5 &   0.07282 & 0.14100 &     6 &     5 \\ \hline
  8 & 0.91882 &        2 &     4 &   0.02832 & 0.06818 &     5 &     2 \\ \hline
  9 & 0.94714 &        2 &     3 &   0.47340 & 0.03986 &     3 &     3 \\ \hline
 1

In [ ]:
print("\\begin{center} \\scalebox{0.7}{")
print(latex_tables[1])
print("}\\end{center}")

\begin{center} \scalebox{0.7}{
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 $\mathbf{l}$ &   $\mathbf{t_\textbf{соб}}$ &  $\mathbf{Type(l)}$ &  $\mathbf{C(l)}$ &  $\mathbf{t_\textbf{остмин}}$ &   $\mathbf{t_\textbf{ожз}}$ &  $\mathbf{j(l)}$ &  $\mathbf{k(l)}$ \\ \hline

 26 & 2.33381 &        2 &     6 &   0.03128 & 0.06319 &     6 &     5 \\ \hline
 27 & 2.36509 &        2 &     5 &   0.18246 & 0.03191 &     8 &     3 \\ \hline
 28 & 2.39700 &        1 &     6 &   0.15054 & 0.14100 &    17 &     3 \\ \hline
 29 & 2.53800 &        1 &     7 &   0.00954 & 0.14100 &    18 &     5 \\ \hline
 30 & 2.54754 &        2 &     6 &   0.31939 & 0.13146 &    14 &     1 \\ \hline
 31 & 2.67900 &        1 &     7 &   0.18793 & 0.14100 &    19 &     1 \\ \hline
 32 & 2.82000 &        1 &     8 &   0.04693 & 0.14100 &    20 &    -1 \\ \hline
 33 & 2.86693 &        2 &     7 &   0.04127 & 0.09407 &    18 &     5 \\ \hline
 34 & 2.90821 &        2 &     6 &   0.20036 & 0.05279 &    12 &     2 \\ \hline
 3

In [ ]:
print("\\begin{center} \\scalebox{0.7}{")
print(latex_tables[2])
print("}\\end{center}")

\begin{center} \scalebox{0.7}{
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 $\mathbf{l}$ &   $\mathbf{t_\textbf{соб}}$ &  $\mathbf{Type(l)}$ &  $\mathbf{C(l)}$ &  $\mathbf{t_\textbf{остмин}}$ &   $\mathbf{t_\textbf{ожз}}$ &  $\mathbf{j(l)}$ &  $\mathbf{k(l)}$ \\ \hline

 51 & 3.94800 &        1 &     5 &   0.66649 & 0.14100 &    28 &     2 \\ \hline
 52 & 4.08900 &        1 &     6 &   0.14883 & 0.14100 &    29 &     5 \\ \hline
 53 & 4.23000 &        1 &     7 &   0.00783 & 0.14100 &    30 &     6 \\ \hline
 54 & 4.23783 &        2 &     6 &   0.37666 & 0.13317 &    29 &     5 \\ \hline
 55 & 4.37100 &        1 &     7 &   0.24349 & 0.14100 &    31 &     5 \\ \hline
 56 & 4.51200 &        1 &     8 &   0.10249 & 0.14100 &    32 &    -1 \\ \hline
 57 & 4.61449 &        2 &     7 &   0.01463 & 0.03851 &    27 &     3 \\ \hline
 58 & 4.62912 &        2 &     6 &   0.28210 & 0.02388 &    31 &     5 \\ \hline
 59 & 4.65300 &        1 &     7 &   0.12850 & 0.14100 &    33 &     5 \\ \hline
 6

In [ ]:
print("\\begin{center} \\scalebox{0.7}{")
print(latex_tables[3])
print("}\\end{center}")

\begin{center} \scalebox{0.7}{
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 $\mathbf{l}$ &   $\mathbf{t_\textbf{соб}}$ &  $\mathbf{Type(l)}$ &  $\mathbf{C(l)}$ &  $\mathbf{t_\textbf{остмин}}$ &   $\mathbf{t_\textbf{ожз}}$ &  $\mathbf{j(l)}$ &  $\mathbf{k(l)}$ \\ \hline

 76 & 6.00869 &        2 &     8 &   0.11043 & 0.05431 &    37 &     4 \\ \hline
 77 & 6.06300 &        1 &     9 &   0.05611 & 0.14100 &    43 &    -1 \\ \hline
 78 & 6.11911 &        2 &     8 &   0.10385 & 0.08489 &    26 &     1 \\ \hline
 79 & 6.20400 &        1 &     9 &   0.01897 & 0.14100 &    44 &    -1 \\ \hline
 80 & 6.22297 &        2 &     8 &   0.35501 & 0.12203 &    40 &     7 \\ \hline
 81 & 6.34500 &        1 &     9 &   0.23297 & 0.14100 &    45 &    -1 \\ \hline
 82 & 6.48600 &        1 &    10 &   0.09197 & 0.14100 &    46 &    -1 \\ \hline
 83 & 6.57797 &        2 &     9 &   0.01272 & 0.04903 &    32 &     3 \\ \hline
 84 & 6.59069 &        2 &     8 &   0.58200 & 0.03631 &    41 &     4 \\ \hline
 8

## Вывод 2

In [ ]:
rdf = pd.DataFrame.from_dict(requests).transpose()
rdf['j'] = rdf['j'].astype(int)

rdf['tw'] = rdf['tw'].round(5)
rdf['te'] = rdf['te'].round(5)
rdf['tq'] = rdf['tq'].round(5)
rdf['ts'] = rdf['ts'].round(5)

rdf['q'] = rdf['q'].astype(int)
rdf['tk'] = rdf['tk'].astype(int)
rdf

,j,tz,q,tq,ts,tw,te,tk
1,1,0.141,0,0.00000,0.14100,1.27954,1.42054,1
2,2,0.282,0,0.00000,0.28200,0.36337,0.64537,2
3,3,0.423,0,0.00000,0.42300,0.52414,0.94714,3
4,4,0.564,0,0.00000,0.56400,4.34722,4.91122,4
5,5,0.705,0,0.00000,0.70500,0.21382,0.91882,2
6,6,0.846,0,0.00000,0.84600,1.48781,2.33381,5
7,7,0.987,0,0.00000,0.98700,0.58523,1.57223,2
8,8,1.128,0,0.00000,1.12800,1.23709,2.36509,3
9,9,1.269,0,0.00000,1.26900,0.75871,2.02771,6
10,10,1.410,0,0.00000,1.41000,0.85718,2.26718,7


In [ ]:
df1, df2, df3 = rdf[:25], rdf[25:50], rdf[50:]
tables = [df1, df2, df3]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-17-7c7b5d23a51f>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &    tz &  q &      tq &      ts &      tw &      te &  tk \\ \hline

 1 & 0.141 &  0 & 0.00000 & 0.14100 & 1.27954 & 1.42054 &   1 \\ \hline
 2 & 0.282 &  0 & 0.00000 & 0.28200 & 0.36337 & 0.64537 &   2 \\ \hline
 3 & 0.423 &  0 & 0.00000 & 0.42300 & 0.52414 & 0.94714 &   3 \\ \hline
 4 & 0.564 &  0 & 0.00000 & 0.56400 & 4.34722 & 4.91122 &   4 \\ \hline
 5 & 0.705 &  0 & 0.00000 & 0.70500 & 0.21382 & 0.91882 &   2 \\ \hline
 6 & 0.846 &  0 & 0.00000 & 0.84600 & 1.48781 & 2.33381 &   5 \\ \hline
 7 & 0.987 &  0 & 0.00000 & 0.98700 & 0.58523 & 1.57223 &   2 \\ \hline
 8 & 1.128 &  0 & 0.00000 & 1.12800 & 1.23709 & 2.36509 &   3 \\ \hline
 9 & 1.269 &  0 & 0.00000 & 1.26900 & 0.75871 & 2.02771 &   6 \\ \hline
10 & 1.410 &  0 & 0.00000 & 1.41000 & 0.85718 & 2.26718 &   7 \\ \hline
11 & 1.551 &  0 & 0.00000 & 1.55100 & 0.20536 & 1.75636 &   1 \\ \hline
12 & 1.692 &  0 & 0.00000 & 1.69200 & 1.21621 & 2.90821 &   2 \\ \hline
13 & 1.833 &  0 & 0.0

In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &    tz &  q &      tq &      ts &      tw &      te &  tk \\ \hline

26 & 3.666 &  0 & 0.00000 & 3.66600 & 2.45311 & 6.11911 &   1 \\ \hline
27 & 3.807 &  0 & 0.00000 & 3.80700 & 0.80749 & 4.61449 &   3 \\ \hline
28 & 3.948 &  0 & 0.00000 & 3.94800 & 4.15710 & 8.10510 &   2 \\ \hline
29 & 4.089 &  0 & 0.00000 & 4.08900 & 0.14883 & 4.23783 &   5 \\ \hline
30 & 4.230 &  0 & 0.00000 & 4.23000 & 3.45746 & 7.68746 &   6 \\ \hline
31 & 4.371 &  0 & 0.00000 & 4.37100 & 0.25812 & 4.62912 &   5 \\ \hline
32 & 4.512 &  1 & 0.10249 & 4.61449 & 1.96348 & 6.57797 &   3 \\ \hline
33 & 4.653 &  0 & 0.00000 & 4.65300 & 0.12850 & 4.78150 &   5 \\ \hline
34 & 4.794 &  0 & 0.00000 & 4.79400 & 0.88871 & 5.68271 &   5 \\ \hline
35 & 4.935 &  0 & 0.00000 & 4.93500 & 0.09390 & 5.02890 &   4 \\ \hline
36 & 5.076 &  0 & 0.00000 & 5.07600 & 0.19419 & 5.27019 &   4 \\ \hline
37 & 5.217 &  1 & 0.05319 & 5.27019 & 0.73850 & 6.00869 &   4 \\ \hline
38 & 5.358 &  1 & 0.0

In [ ]:
print(latex_tables[2])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &    tz &  q &       tq &       ts &       tw &       te &  tk \\ \hline

51 & 7.191 &  4 &  0.53489 &  7.72589 &  1.07465 &  8.80053 &   5 \\ \hline
52 & 7.332 &  5 &  0.43079 &  7.76279 &  0.62731 &  8.39010 &   1 \\ \hline
53 & 7.473 &  6 & -1.00000 & -1.00000 & -1.00000 & -1.00000 &  -1 \\ \hline
54 & 7.614 &  6 & -1.00000 & -1.00000 & -1.00000 & -1.00000 &  -1 \\ \hline
55 & 7.755 &  4 & -1.00000 & -1.00000 & -1.00000 & -1.00000 &  -1 \\ \hline

\end{tabular}



## Вывод 3

In [ ]:
final_t = adf["t_соб"][99]
final_t

7.76279

In [ ]:
ddf = pd.DataFrame([[device.device_id, device.processed, device.was_busy, final_t-device.was_busy, (final_t-device.was_busy)/final_t] for device in devices],
                   columns=['k', 'N(k)', 't_зан', 't_np', "Delta"])
ddf.loc[len(ddf.index)] = [None, sum(ddf["N(k)"]), sum(ddf["N(k)"])/n,  sum(ddf["t_np"])/n,  sum(ddf["Delta"])/n]

ddf['k'] = ddf['k'].astype('Int64')
ddf['N(k)'] = ddf['N(k)'].astype(int)
ddf['t_зан'] = ddf['t_зан'].round(5)
ddf['t_np'] = ddf['t_np'].round(5)
ddf['Delta'] = ddf['Delta'].round(5)

ddf

,k,N(k),t_зан,t_np,Delta
0,1,10,7.18261,0.58018,0.07474
1,2,6,7.11952,0.64327,0.08287
2,3,6,7.12354,0.63925,0.08235
3,4,8,7.12792,0.63487,0.08178
4,5,12,6.12421,1.63858,0.21108
5,6,5,5.89732,1.86547,0.24031
6,7,5,6.35279,1.41000,0.18164
7,<NA>,52,7.42857,1.05880,0.13639


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')
    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
latex_table = latex_with_lines(ddf)

<ipython-input-23-24eb57417011>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_table)

\begin{tabular}{|c|c|c|c|c|}
\hline
   k &  N(k) &   t\_зан &    t\_np &   Delta \\ \hline

   1 &    10 & 7.18261 & 0.58018 & 0.07474 \\ \hline
   2 &     6 & 7.11952 & 0.64327 & 0.08287 \\ \hline
   3 &     6 & 7.12354 & 0.63925 & 0.08235 \\ \hline
   4 &     8 & 7.12792 & 0.63487 & 0.08178 \\ \hline
   5 &    12 & 6.12421 & 1.63858 & 0.21108 \\ \hline
   6 &     5 & 5.89732 & 1.86547 & 0.24031 \\ \hline
   7 &     5 & 6.35279 & 1.41000 & 0.18164 \\ \hline
  &    52 & 7.42857 & 1.05880 & 0.13639 \\ \hline

\end{tabular}



## Вывод 4

In [ ]:
adf['C(l)'].value_counts().sort_index().index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [ ]:
R = adf['C(l)'].value_counts().sort_index().values
if adf['C(l)'].value_counts().sort_index().index[0] != 0:
  R = np.insert(R, 0, 0)
R

array([ 0,  1,  1,  3,  6,  6, 19, 26, 15,  7,  6,  5,  3,  2])

In [ ]:
sum([r/100 for r in R])

1.0000000000000002

In [ ]:
s = final_t

T = [0]*(len(R))
D = [0]*len(R)
cur_state = 0
last_time = 0

for e in actions:
  _st = e[3]
  if _st != cur_state:
    # print(_st, e[1])
    T[cur_state] += e[1] - last_time
    last_time, cur_state = e[1], e[3]

for i in range(len(T)):
  D[i] = T[i]/s
print(T)
print(s, D, sum(D))

[0.141, 0.141, 0.14099999999999996, 0.2404863378005042, 0.45607265198852287, 0.4674981394883786, 1.6084653619629323, 2.0047375221910997, 0.976570823260906, 0.49919451643886337, 0.37068269076462457, 0.360622818886994, 0.3324294054193846, 0.023029525102569615]
7.76279 [0.018163572633035287, 0.018163572633035287, 0.018163572633035283, 0.03097936924746183, 0.05875112581797561, 0.06022295327947537, 0.20720196758677387, 0.2582496141453137, 0.125801525387252, 0.06430606991028527, 0.047751219698668204, 0.04645531038286415, 0.042823444331146994, 0.0029666556872683164] 0.9999999733735913


In [ ]:
states = np.array(adf['C(l)'].value_counts().sort_index().index)
if states[0] != 0:
  states = np.insert(states, 0, 0)
states

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [ ]:
sdf = pd.DataFrame({
    'Состояние': states,
    'R': R
})
sdf['v'] = sdf["R"]/100
sdf['T'] = T
sdf['Delta'] = D

sdf.loc[len(sdf.index)] = [None, sum(sdf["R"]),  sum(sdf["v"]), sum(sdf["T"]), sum(sdf["Delta"])]

sdf['Состояние'] = sdf['Состояние'].astype("Int64")
sdf['R'] = sdf['R'].astype(int)
sdf['T'] = sdf['T'].round(5)
sdf['Delta'] = sdf['Delta'].round(5)

sdf

,Состояние,R,v,T,Delta
0,0,0,0.00,0.14100,0.01816
1,1,1,0.01,0.14100,0.01816
2,2,1,0.01,0.14100,0.01816
3,3,3,0.03,0.24049,0.03098
4,4,6,0.06,0.45607,0.05875
5,5,6,0.06,0.46750,0.06022
6,6,19,0.19,1.60847,0.20720
7,7,26,0.26,2.00474,0.25825
8,8,15,0.15,0.97657,0.12580
9,9,7,0.07,0.49919,0.06431


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
tables = [sdf[:25], sdf[25:]]
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-31-e048387fbaa6>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


!!!! ПРОВЕРЬ КОЛ-ВО СОСТОЯНИЙ ПРЕЖДЕ ЧЕМ РЕЗАТЬ ИСХОДНУЮ ТАБЛИЦУ

In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 Состояние &   R &    v &       T &   Delta \\ \hline

         0 &   0 & 0.00 & 0.14100 & 0.01816 \\ \hline
         1 &   1 & 0.01 & 0.14100 & 0.01816 \\ \hline
         2 &   1 & 0.01 & 0.14100 & 0.01816 \\ \hline
         3 &   3 & 0.03 & 0.24049 & 0.03098 \\ \hline
         4 &   6 & 0.06 & 0.45607 & 0.05875 \\ \hline
         5 &   6 & 0.06 & 0.46750 & 0.06022 \\ \hline
         6 &  19 & 0.19 & 1.60847 & 0.20720 \\ \hline
         7 &  26 & 0.26 & 2.00474 & 0.25825 \\ \hline
         8 &  15 & 0.15 & 0.97657 & 0.12580 \\ \hline
         9 &   7 & 0.07 & 0.49919 & 0.06431 \\ \hline
        10 &   6 & 0.06 & 0.37068 & 0.04775 \\ \hline
        11 &   5 & 0.05 & 0.36062 & 0.04646 \\ \hline
        12 &   3 & 0.03 & 0.33243 & 0.04282 \\ \hline
        13 &   2 & 0.02 & 0.02303 & 0.00297 \\ \hline
        & 100 & 1.00 & 7.76279 & 1.00000 \\ \hline

\end{tabular}



In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
Empty DataFrame
Columns: Index(['Состояние', 'R', 'v', 'T', 'Delta'], dtype='object')
Index: Int64Index([], dtype='int64') \\ \hline

\end{tabular}



## Вывод 5

In [ ]:
print(len(rdf)) #число заявок

55


In [ ]:
rdf[(rdf["te"] > adf["t_соб"][99])] # Эти заявки не завершились на последнем событии или стоят в очереди

,j,tz,q,tq,ts,tw,te,tk
28,28,3.948,0,0.00000,3.94800,4.15710,8.10510,2
43,43,6.063,2,0.15997,6.22297,1.58415,7.80712,7
44,44,6.204,2,0.37397,6.57797,1.26549,7.84346,3
47,47,6.627,2,0.49764,7.12464,2.31840,9.44304,4
50,50,7.050,4,0.63746,7.68746,0.80091,8.48837,6
51,51,7.191,4,0.53489,7.72589,1.07465,8.80053,5
52,52,7.332,5,0.43079,7.76279,0.62731,8.39010,1


In [ ]:
obsl = len(rdf[(rdf["tk"] != -1) & (rdf["te"] <= adf["t_соб"][99])]) # Обработанные заявки
obsl

45

In [ ]:
sum(zl)/100 # среднее число заявок

7.22

In [ ]:
rdf["tq"][rdf["tq"] >= 0].sum()/obsl # среднее пребывание в очереди

0.1307431111111111

In [ ]:
n

7

In [ ]:
(rdf["te"][rdf["tq"] >= 0] - rdf["tz"][rdf["tq"] >= 0]).sum()/obsl # среднее пребывание в СМО

1.2744342222222225

Теорчасть

# Система массового обслуживания (M|M|n)

In [ ]:
class Device:
    def __init__(self, device_id):
        self.device_id = device_id
        self.is_busy = False
        self.finish_time = None
        self.cur_request = None
        self.processing_time = None
        self.processed = 0
        self.was_busy = 0


    def finish(self, time):
      global requests
      if not self.is_busy: return
      print(f"DEVICE {self.device_id} HAS UNFINISHED JOB!")
      self.was_busy += time - requests[self.cur_request]["ts"]


    def new_request(self, request, write=True, override=False):
        global counter
        global actions
        global requests
        global state
        global left_time
        global next_query_deadline
        global next_next
        global min_finish
        self.is_busy = True
        self.processed += 1

        time = next_query_deadline if not override else override

        # Записать время и событие принятия заявки
        print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}')

        processing_time = np.random.exponential(1/mu)
        self.processing_time = processing_time
        if write:
          counter += 1
          actions.append([
            counter, #number
            time, # time recieved
            1, # type
            state, # state
            processing_time if processing_time < min_finish-time else min_finish-time, #min time left of work
            next_next, # next query in
            request, # request num
            self.device_id # device number
          ])

        self.finish_time = time + processing_time
        self.cur_request = request

        requests[request].update({
          "tq": time - requests[request]["tz"], # queue time
          "ts": time,
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
          "tk": self.device_id # who worked on request
        })


    def process_request(self):
        global counter
        global actions
        global state
        global left_time
        global next_query_deadline
        global min_device
        global devices
        global queue

        min_finish = np.inf
        min_device = None
        free_device = None

        for device in devices:
          if device.is_busy and device.finish_time < min_finish and device.device_id != self.device_id:
            min_finish = device.finish_time
            min_device = device
          elif not device.is_busy and free_device is None and device.device_id != self.device_id: free_device = device

        # Записать время и событие завершения обработки
        counter += 1
        state -= 1
        self.was_busy += self.processing_time

        print(f'{self.finish_time}: Устройство {self.device_id} обработало заявку {self.cur_request}, {counter}')

        requests[self.cur_request].update({
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
        })

        # requests[self.cur_request].update({
        # })

        # есть ли новые заявки в очереди, добавить новый реквест
        if len(queue) != 0 and counter <= 100:
            self.processed += 1
            actions.append([
              counter, #number
              self.finish_time, # time recieved
              2, # type
              state, # state
              self.processing_time if self.processing_time < min_finish-self.finish_time else min_finish-self.finish_time, #min time left of work
              next_query_deadline - self.finish_time, # next query in
              self.cur_request, # request num
              self.device_id # device number
            ])

            request = queue.pop(0)
            time = self.finish_time

            # Записать время и событие принятия заявки
            print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}')

            processing_time = np.random.exponential(1/mu)
            self.processing_time = processing_time

            self.finish_time = time + processing_time
            self.cur_request = request

            requests[request].update({
              "tq": time - requests[request]["tz"], # queue time
              "ts": time,
              "tw": self.processing_time, # for how long worked
              "te": self.finish_time, # when finished working
              "tk": self.device_id # who worked on request
            })
        elif counter <= 100:
          # print(len(queue))
          actions.append([
            counter, #number
            self.finish_time, # time recieved
            2, # type
            state, # state
            -1 if min_device is None else min_finish-self.finish_time,
            next_query_deadline - self.finish_time, # next query in
            self.cur_request, # request num
            self.device_id # device number
          ])
          self.finish_time = None
          self.cur_request = None
          self.is_busy = False

In [ ]:
counter = 0 # actions
request = 0 # queries
state = 0 # current state of query system
next_query_deadline = np.random.exponential(1/lam) # time when the new query appears
next_next =  np.random.exponential(1/lam)

queue = [] # queries queue # .pop(0)
requests = {} # log of requests
actions = [] # log of actions

zl = []

devices = [Device(i+1) for i in range(n)]
# watcher
while counter < 100:
  min_finish = np.inf
  min_device = None
  free_device = None

  # get device that will end sooner than others
  for device in devices:
    if device.is_busy and device.finish_time < min_finish:
      min_finish = device.finish_time
      min_device = device
    elif not device.is_busy and free_device is None: free_device = device

  # is there any query that fires sooner?
  if next_query_deadline < min_finish:
    request += 1
    if len(zl): zl.append(zl[-1]+1)
    else: zl.append(1)

    if free_device is None:
      # all devices are busy. Add to queue
      counter += 1
      state += 1
      print(f'{next_query_deadline}: Новая заявка {request} встала в очередь, {counter}')
      actions.append([
            counter, #number
            next_query_deadline, # time recieved
            1, # type
            state, # state
            -1 if min_device is None else min_finish-next_query_deadline, #min time left of work
            next_next, # next query in
            request, # request num
            -1 # device number
        ])
      queue.append(request)

      requests[request] = {
          "j": request,
          "tz": next_query_deadline, # appeared
          "q": len(queue), # queue pos
          "tq": -1, # queue time
          "ts": -1, # start of working on request
          "tw": -1, # for how long worked
          "te": -1, # when finished working
          "tk": -1 # who worked on request
      }
    else:
      # At least one device is free
      state += 1
      requests[request] = {
          "j": request,
          "tz": next_query_deadline, # appeared
          "q": 0, # queue pos
          "tq": 0, # queue time
          "ts": next_query_deadline, # start of working on request
          "tw": 0, # for how long worked
          "te": -1, # when finished working
          "tk": free_device.device_id # who worked on request
      }
      free_device.new_request(request)
    next_query_deadline +=  next_next
    next_next = np.random.exponential(1/lam)
  else:
    # Device processes query sooner
    zl.append(zl[-1]-1)
    min_device.process_request()


0.09702883545326361: Устройство 1 приняло заявку 1, 0
0.2348466727106835: Устройство 2 приняло заявку 2, 1
0.24117763058420835: Устройство 3 приняло заявку 3, 2
0.27876406903749923: Устройство 4 приняло заявку 4, 3
0.31344117848421543: Устройство 1 обработало заявку 1, 5
0.34868575260934626: Устройство 1 приняло заявку 5, 5
0.3700371660770373: Устройство 5 приняло заявку 6, 6
0.4407421309972932: Устройство 6 приняло заявку 7, 7
0.6220257698767097: Устройство 7 приняло заявку 8, 8
0.6314616451518194: Устройство 3 обработало заявку 3, 10
0.7633199974873968: Устройство 6 обработало заявку 7, 11
0.847872060108768: Устройство 2 обработало заявку 2, 12
0.8698117821101901: Устройство 4 обработало заявку 4, 13
0.9128289490199026: Устройство 1 обработало заявку 5, 14
1.0294614489659983: Устройство 1 приняло заявку 9, 14
1.4207546690087698: Устройство 2 приняло заявку 10, 15
1.470609467398337: Устройство 7 обработало заявку 8, 17
1.4937616556830595: Устройство 3 приняло заявку 11, 17
1.504706817

In [ ]:
for device in devices:
  device.finish(actions[-1][1])

DEVICE 1 HAS UNFINISHED JOB!
DEVICE 3 HAS UNFINISHED JOB!
DEVICE 5 HAS UNFINISHED JOB!
DEVICE 6 HAS UNFINISHED JOB!


In [ ]:
with open('actions2.pickle', 'wb') as handle:
    pickle.dump(actions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('requests2pickle', 'wb') as handle:
    pickle.dump(requests, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('devices2.pickle', 'wb') as handle:
    pickle.dump(devices, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Вывод 1

In [ ]:
adf = pd.DataFrame(actions, columns=['l', 't_соб', 'Type(l)', 'C(l)', 't_остмин', "t_ожз", "j(l)", "k(l)"])
adf['t_соб'] = adf['t_соб'].round(5)
adf['t_остмин'] = adf['t_остмин'].round(5)
adf['t_ожз'] = adf['t_ожз'].round(5)

adf['k(l)'] = adf['k(l)'].astype(int)

adf[:100]

,l,t_соб,Type(l),C(l),t_остмин,t_ожз,j(l),k(l)
0,1,0.09703,1,1,0.21641,0.13782,1,1
1,2,0.23485,1,2,0.07859,0.00633,2,2
2,3,0.24118,1,3,0.07226,0.03759,3,3
3,4,0.27876,1,4,0.03468,0.06992,4,4
4,5,0.31344,2,3,0.31802,0.03524,1,1
...,...,...,...,...,...,...,...,...
95,96,8.07393,1,4,0.16812,0.10979,50,4
96,97,8.18371,1,5,0.05833,0.04793,51,5
97,98,8.23164,1,6,0.01040,0.35342,52,6
98,99,8.24204,2,5,0.31848,0.34301,50,4


In [ ]:
df1, df2, df3, df4 = np.array_split(adf, 4)
tables = [df1, df2, df3, df4]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-47-7c7b5d23a51f>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

 1 & 0.09703 &        1 &     1 &   0.21641 & 0.13782 &     1 &     1 \\ \hline
 2 & 0.23485 &        1 &     2 &   0.07859 & 0.00633 &     2 &     2 \\ \hline
 3 & 0.24118 &        1 &     3 &   0.07226 & 0.03759 &     3 &     3 \\ \hline
 4 & 0.27876 &        1 &     4 &   0.03468 & 0.06992 &     4 &     4 \\ \hline
 5 & 0.31344 &        2 &     3 &   0.31802 & 0.03524 &     1 &     1 \\ \hline
 6 & 0.34869 &        1 &     4 &   0.28278 & 0.02135 &     5 &     1 \\ \hline
 7 & 0.37004 &        1 &     5 &   0.26142 & 0.07070 &     6 &     5 \\ \hline
 8 & 0.44074 &        1 &     6 &   0.19072 & 0.18128 &     7 &     6 \\ \hline
 9 & 0.62203 &        1 &     7 &   0.00944 & 0.40744 &     8 &     7 \\ \hline
10 & 0.63146 &        2 &     6 &   0.13186 & 0.39800 &     3 &     3 \\ \hline
11 & 0.76332 &        2 &     5 &   0.08455 & 0.26614 &     7 &     6 \\ \

In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

26 & 2.08118 &        2 &     6 &   0.06246 & 0.02248 &     6 &     5 \\ \hline
27 & 2.10366 &        1 &     7 &   0.03998 & 0.11849 &    17 &     5 \\ \hline
28 & 2.14364 &        2 &     6 &   0.17817 & 0.07851 &    12 &     4 \\ \hline
29 & 2.22215 &        1 &     7 &   0.09966 & 0.16815 &    18 &     4 \\ \hline
30 & 2.32181 &        2 &     6 &   0.23088 & 0.06849 &    16 &     1 \\ \hline
31 & 2.39029 &        1 &     7 &   0.14881 & 0.48649 &    19 &     1 \\ \hline
32 & 2.53911 &        2 &     6 &   0.01358 & 0.33767 &    19 &     1 \\ \hline
33 & 2.55269 &        2 &     5 &   0.10635 & 0.32409 &    17 &     5 \\ \hline
34 & 2.65904 &        2 &     4 &   0.14623 & 0.21774 &    13 &     6 \\ \hline
35 & 2.80527 &        2 &     3 &   0.16364 & 0.07151 &    15 &     7 \\ \hline
36 & 2.87678 &        1 &     4 &   0.09213 & 0.07809 &    20 &     1 \\ \

In [ ]:
print(latex_tables[2])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

51 & 3.99615 &        1 &     5 &   0.02747 & 0.01572 &    28 &     4 \\ \hline
52 & 4.01186 &        1 &     6 &   0.01175 & 0.10626 &    29 &     5 \\ \hline
53 & 4.02362 &        2 &     5 &   0.06832 & 0.09451 &    28 &     4 \\ \hline
54 & 4.09194 &        2 &     4 &   0.03855 & 0.02618 &    25 &     2 \\ \hline
55 & 4.11813 &        1 &     5 &   0.01237 & 0.05745 &    30 &     2 \\ \hline
56 & 4.13050 &        2 &     4 &   0.02439 & 0.04508 &    27 &     3 \\ \hline
57 & 4.15489 &        2 &     3 &   0.21366 & 0.02069 &    23 &     6 \\ \hline
58 & 4.17557 &        1 &     4 &   0.19297 & 0.08153 &    31 &     3 \\ \hline
59 & 4.25710 &        1 &     5 &   0.11145 & 0.05213 &    32 &     4 \\ \hline
60 & 4.30923 &        1 &     6 &   0.05932 & 0.00707 &    33 &     6 \\ \hline
61 & 4.31630 &        1 &     7 &   0.05225 & 0.29746 &    34 &     7 \\ \

In [ ]:
print(latex_tables[3])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
  l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

 76 & 5.15842 &        2 &     4 &   0.60451 & 0.25314 &    39 &     3 \\ \hline
 77 & 5.41156 &        1 &     5 &   0.35137 & 0.30152 &    41 &     3 \\ \hline
 78 & 5.71308 &        1 &     6 &   0.04985 & 0.19519 &    42 &     5 \\ \hline
 79 & 5.76293 &        2 &     5 &   0.16079 & 0.14534 &    34 &     7 \\ \hline
 80 & 5.90827 &        1 &     6 &   0.01545 & 0.03656 &    43 &     6 \\ \hline
 81 & 5.92372 &        2 &     5 &   0.00854 & 0.02111 &    37 &     1 \\ \hline
 82 & 5.93227 &        2 &     4 &   0.04519 & 0.01256 &    41 &     3 \\ \hline
 83 & 5.94483 &        1 &     5 &   0.03263 & 1.15555 &    44 &     1 \\ \hline
 84 & 5.97746 &        2 &     4 &   0.10817 & 1.12292 &    38 &     2 \\ \hline
 85 & 6.08562 &        2 &     3 &   0.03243 & 1.01475 &    44 &     1 \\ \hline
 86 & 6.11806 &        2 &     2 &   0.52393 & 0.98232 &    43 

## Вывод 2

In [ ]:
rdf = pd.DataFrame.from_dict(requests).transpose()
rdf['j'] = rdf['j'].astype(int)

rdf['tw'] = rdf['tw'].round(5)
rdf['te'] = rdf['te'].round(5)
rdf['tq'] = rdf['tq'].round(5)
rdf['ts'] = rdf['ts'].round(5)
rdf['tz'] = rdf['tz'].round(5)

rdf['q'] = rdf['q'].astype(int)
rdf['tk'] = rdf['tk'].astype(int)
rdf

,j,tz,q,tq,ts,tw,te,tk
1,1,0.09703,0,0.00000,0.09703,0.21641,0.31344,1
2,2,0.23485,0,0.00000,0.23485,0.61303,0.84787,2
3,3,0.24118,0,0.00000,0.24118,0.39028,0.63146,3
4,4,0.27876,0,0.00000,0.27876,0.59105,0.86981,4
5,5,0.34869,0,0.00000,0.34869,0.56414,0.91283,1
6,6,0.37004,0,0.00000,0.37004,1.71114,2.08118,5
7,7,0.44074,0,0.00000,0.44074,0.32258,0.76332,6
8,8,0.62203,0,0.00000,0.62203,0.84858,1.47061,7
9,9,1.02946,0,0.00000,1.02946,1.03225,2.06171,1
10,10,1.42075,0,0.00000,1.42075,0.38185,1.80261,2


In [ ]:
df1, df2, df3 = rdf[:25], rdf[25:50], rdf[50:]
tables = [df1, df2, df3]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-54-7c7b5d23a51f>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &      tq &      ts &      tw &      te &  tk \\ \hline

 1 & 0.09703 &  0 & 0.00000 & 0.09703 & 0.21641 & 0.31344 &   1 \\ \hline
 2 & 0.23485 &  0 & 0.00000 & 0.23485 & 0.61303 & 0.84787 &   2 \\ \hline
 3 & 0.24118 &  0 & 0.00000 & 0.24118 & 0.39028 & 0.63146 &   3 \\ \hline
 4 & 0.27876 &  0 & 0.00000 & 0.27876 & 0.59105 & 0.86981 &   4 \\ \hline
 5 & 0.34869 &  0 & 0.00000 & 0.34869 & 0.56414 & 0.91283 &   1 \\ \hline
 6 & 0.37004 &  0 & 0.00000 & 0.37004 & 1.71114 & 2.08118 &   5 \\ \hline
 7 & 0.44074 &  0 & 0.00000 & 0.44074 & 0.32258 & 0.76332 &   6 \\ \hline
 8 & 0.62203 &  0 & 0.00000 & 0.62203 & 0.84858 & 1.47061 &   7 \\ \hline
 9 & 1.02946 &  0 & 0.00000 & 1.02946 & 1.03225 & 2.06171 &   1 \\ \hline
10 & 1.42075 &  0 & 0.00000 & 1.42075 & 0.38185 & 1.80261 &   2 \\ \hline
11 & 1.49376 &  0 & 0.00000 & 1.49376 & 1.93485 & 3.42861 &   3 \\ \hline
12 & 1.50471 &  0 & 0.00000 & 1.50471 & 0.63893 & 2.14364 &   4 \\ \h

In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &  tq &      ts &      tw &      te &  tk \\ \hline

26 & 3.76572 &  0 & 0.0 & 3.76572 & 0.60282 & 4.36855 &   1 \\ \hline
27 & 3.99052 &  0 & 0.0 & 3.99052 & 0.13997 & 4.13050 &   3 \\ \hline
28 & 3.99615 &  0 & 0.0 & 3.99615 & 0.02747 & 4.02362 &   4 \\ \hline
29 & 4.01186 &  0 & 0.0 & 4.01186 & 0.72982 & 4.74169 &   5 \\ \hline
30 & 4.11813 &  0 & 0.0 & 4.11813 & 0.25491 & 4.37303 &   2 \\ \hline
31 & 4.17557 &  0 & 0.0 & 4.17557 & 0.54319 & 4.71876 &   3 \\ \hline
32 & 4.25710 &  0 & 0.0 & 4.25710 & 0.22673 & 4.48383 &   4 \\ \hline
33 & 4.30923 &  0 & 0.0 & 4.30923 & 0.08651 & 4.39574 &   6 \\ \hline
34 & 4.31630 &  0 & 0.0 & 4.31630 & 1.44663 & 5.76293 &   7 \\ \hline
35 & 4.61376 &  0 & 0.0 & 4.61376 & 0.20061 & 4.81437 &   1 \\ \hline
36 & 4.77367 &  0 & 0.0 & 4.77367 & 0.02075 & 4.79442 &   2 \\ \hline
37 & 4.81959 &  0 & 0.0 & 4.81959 & 1.10413 & 5.92372 &   1 \\ \hline
38 & 5.00479 &  0 & 0.0 & 5.00479 & 0.97267 & 5

In [ ]:
print(latex_tables[2])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &  tq &      ts &      tw &      te &  tk \\ \hline

51 & 8.18371 &  0 & 0.0 & 8.18371 & 1.36062 & 9.54433 &   5 \\ \hline
52 & 8.23164 &  0 & 0.0 & 8.23164 & 1.50625 & 9.73789 &   6 \\ \hline

\end{tabular}



## Вывод 3

In [ ]:
final_t = adf["t_соб"][99]
final_t

8.56053

In [ ]:
ddf = pd.DataFrame([[device.device_id, device.processed, device.was_busy, final_t-device.was_busy, (final_t-device.was_busy)/final_t] for device in devices],
                   columns=['k', 'N(k)', 't_зан', 't_np', "Delta"])
ddf.loc[len(ddf.index)] = [None, sum(ddf["N(k)"]), sum(ddf["N(k)"])/n,  sum(ddf["t_np"])/n,  sum(ddf["Delta"])/n]

ddf['k'] = ddf['k'].astype("Int64")
ddf['N(k)'] = ddf['N(k)'].astype(int)
ddf['t_зан'] = ddf['t_зан'].round(5)
ddf['t_np'] = ddf['t_np'].round(5)
ddf['Delta'] = ddf['Delta'].round(5)

ddf

,k,N(k),t_зан,t_np,Delta
0,1,11,6.12788,2.43265,0.28417
1,2,11,5.68148,2.87905,0.33632
2,3,7,4.30092,4.25961,0.49759
3,4,7,4.85628,3.70425,0.43271
4,5,6,5.17912,3.38141,0.39500
5,6,6,3.15443,5.40610,0.63151
6,7,4,3.50321,5.05732,0.59077
7,<NA>,52,7.42857,3.87434,0.45258


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')
    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
latex_table = latex_with_lines(ddf)

<ipython-input-60-24eb57417011>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_table)

\begin{tabular}{|c|c|c|c|c|}
\hline
   k &  N(k) &   t\_зан &    t\_np &   Delta \\ \hline

   1 &    11 & 6.12788 & 2.43265 & 0.28417 \\ \hline
   2 &    11 & 5.68148 & 2.87905 & 0.33632 \\ \hline
   3 &     7 & 4.30092 & 4.25961 & 0.49759 \\ \hline
   4 &     7 & 4.85628 & 3.70425 & 0.43271 \\ \hline
   5 &     6 & 5.17912 & 3.38141 & 0.39500 \\ \hline
   6 &     6 & 3.15443 & 5.40610 & 0.63151 \\ \hline
   7 &     4 & 3.50321 & 5.05732 & 0.59077 \\ \hline
  &    52 & 7.42857 & 3.87434 & 0.45258 \\ \hline

\end{tabular}



## Вывод 4

In [ ]:
adf['C(l)'].value_counts().sort_index().index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [ ]:
R = adf['C(l)'].value_counts().sort_index().values
if adf['C(l)'].value_counts().sort_index().index[0] != 0:
  R = np.insert(R, 0, 0)
R

array([ 1,  6,  9, 15, 22, 20, 17,  9,  1])

In [ ]:
sum([r/100 for r in R])

1.0

In [ ]:
s = final_t

T = [0]*(len(R))
D = [0]*len(R)
cur_state = 0
last_time = 0

for e in actions:
  _st = e[3]
  if _st != cur_state:
    # print(_st, e[1])
    T[cur_state] += e[1] - last_time
    last_time, cur_state = e[1], e[3]

for i in range(len(T)):
  D[i] = T[i]/s
print(T)
print(s, D, sum(D))

[0.1568580938723761, 0.8220621215516246, 1.438828022996734, 1.3516892197829613, 1.4326008759226236, 1.5158265999277014, 1.1596535637554637, 0.484079901722552, 0.19893012571554003]
8.56053 [0.018323409166532457, 0.09602934883139533, 0.1680769792287083, 0.15789784274840007, 0.1673495538153156, 0.17707158317624042, 0.13546515972205736, 0.05654788917538423, 0.023238061862471137] 0.9999998277265048


In [ ]:
states = np.array(adf['C(l)'].value_counts().sort_index().index)
if states[0] != 0:
  states = np.insert(states, 0, 0)
states

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
def calculate_P0(lam, mu, n, m):
  rho = lam/mu
  nu = rho/n
  return 6.5120014164921E-7 #
  return np.sum([(rho**k)/(factorial(k)) for k in range(n)] + [((rho**n)/factorial(n)) * np.sum([nu**k for k in range(m+1)])])**-1

def calculate_Pi(lam, mu, n, m, i):
    rho = lam/mu
    return (rho**i/factorial(i)) * calculate_P0(lam ,mu, n, m)


p0 = calculate_P0(lam, mu, n, m)
Pi = calculate_Pi(lam, mu, n, m, 19)
print(p0, Pi)

6.5120014164921e-07 4.737896164426495e-08


In [ ]:
np.sum([calculate_Pi(lam, mu, n, m, i) for i in range(20)])

0.0006509382265091893

In [ ]:
sdf = pd.DataFrame({
    'Состояние': states,
    'r_i': [calculate_Pi(lam, mu, n, m, i) for i in range(len(R))],
    'R': R
})
sdf['v'] = sdf["R"]/100
sdf['T'] = T
sdf['Delta'] = D

sdf.loc[len(sdf.index)] = [None, sum(sdf["r_i"]), sum(sdf["R"]),  sum(sdf["v"]), sum(sdf["T"]), sum(sdf["Delta"])]

sdf['Состояние'] = sdf['Состояние'].astype("Int64")
sdf['r_i'] = sdf['r_i'].round(5)
sdf['R'] = sdf['R'].astype(int)
sdf['T'] = sdf['T'].round(5)
sdf['Delta'] = sdf['Delta'].round(5)

sdf

,Состояние,r_i,R,v,T,Delta
0,0,0.00000,1,0.01,0.15686,0.01832
1,1,0.00000,6,0.06,0.82206,0.09603
2,2,0.00002,9,0.09,1.43883,0.16808
3,3,0.00004,15,0.15,1.35169,0.15790
4,4,0.00006,22,0.22,1.43260,0.16735
5,5,0.00009,20,0.20,1.51583,0.17707
6,6,0.00010,17,0.17,1.15965,0.13547
7,7,0.00010,9,0.09,0.48408,0.05655
8,8,0.00008,1,0.01,0.19893,0.02324
9,<NA>,0.00048,100,1.00,8.56053,1.00000


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')
    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
tables = [sdf[:25], sdf[25:]]
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-70-6383ca4c28e7>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


!!!! ПРОВЕРЬ КОЛ-ВО СОСТОЯНИЙ ПРЕЖДЕ ЧЕМ РЕЗАТЬ ИСХОДНУЮ ТАБЛИЦУ

In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 Состояние &     r\_i &   R &    v &       T &   Delta \\ \hline

         0 & 0.00000 &   1 & 0.01 & 0.15686 & 0.01832 \\ \hline
         1 & 0.00000 &   6 & 0.06 & 0.82206 & 0.09603 \\ \hline
         2 & 0.00002 &   9 & 0.09 & 1.43883 & 0.16808 \\ \hline
         3 & 0.00004 &  15 & 0.15 & 1.35169 & 0.15790 \\ \hline
         4 & 0.00006 &  22 & 0.22 & 1.43260 & 0.16735 \\ \hline
         5 & 0.00009 &  20 & 0.20 & 1.51583 & 0.17707 \\ \hline
         6 & 0.00010 &  17 & 0.17 & 1.15965 & 0.13547 \\ \hline
         7 & 0.00010 &   9 & 0.09 & 0.48408 & 0.05655 \\ \hline
         8 & 0.00008 &   1 & 0.01 & 0.19893 & 0.02324 \\ \hline
        & 0.00048 & 100 & 1.00 & 8.56053 & 1.00000 \\ \hline

\end{tabular}



In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
Empty DataFrame
Columns: Index(['Состояние', 'r\_i', 'R', 'v', 'T', 'Delta'], dtype='object')
Index: Int64Index([], dtype='int64') \\ \hline

\end{tabular}



## Вывод 5

In [ ]:
print(len(rdf)) #число заявок

52


In [ ]:
rdf[(rdf["te"] > adf["t_соб"][99]) | (rdf["tk"] == -1)] # Эти заявки не завершились на последнем событии или стоят в очереди

,j,tz,q,tq,ts,tw,te,tk
45,45,7.10037,0,0.0,7.10037,2.06401,9.16438,1
49,49,7.93565,0,0.0,7.93565,1.76104,9.69669,3
51,51,8.18371,0,0.0,8.18371,1.36062,9.54433,5
52,52,8.23164,0,0.0,8.23164,1.50625,9.73789,6


In [ ]:
obsl = len(rdf[(rdf["tk"] != -1) & (rdf["te"] <= adf["t_соб"][99])]) # Обработанные заявки
obsl

48

In [ ]:
sum(zl)/100 # среднее число заявок

4.3

In [ ]:
rdf["tq"][rdf["tq"] >= 0].sum()/obsl # среднее пребывание в очереди

0.004144375

In [ ]:
np.round(0.036249099470435434, 5)

0.03625

In [ ]:
(rdf["te"][rdf["tq"] >= 0] - rdf["tz"][rdf["tq"] >= 0]).sum()/obsl # среднее пребывание в СМО

0.7688212500000001

In [ ]:
0.03625+3.20156

3.23781

In [ ]:
adf["C(l)"]

0     1
1     2
2     3
3     4
4     3
     ..
95    4
96    5
97    6
98    5
99    4
Name: C(l), Length: 100, dtype: int64

In [ ]:
# Среднее число занятых приборов ЭТО НЕ ТЕОРЕТИЧЕСКИЕ
# adf["C(l)"].copy().apply(lambda x: x if x < n else n).sum()/100

In [ ]:
# Средняя длина очереди
# adf["C(l)"].copy().apply(lambda x: x-n if x-n >= 0 else 0).sum()/100

In [ ]:
p = lam/(mu)
p

6.907390084190833

In [ ]:
P_0 = 1 / (sum((lam/mu)**k / math.factorial(k) for k in range(n)) +
               ((lam/mu)**n / (math.factorial(n) * (1 - p))))
L = lam * mu * (p * ((lam/mu)**n) * P_0) / ((n * mu - lam)**2) + lam / mu
L # средняя теор занятых приборов

9523734.32677922

In [ ]:
L_q = (p * n * math.pow(p, n)) / (math.factorial(n) * (1 - p) * (1 - math.pow(p, n+1)))
for i in range(n):
    L_q += (math.pow(p * n, i) / math.factorial(i))
L_q = p * n - L_q
L_q # Средняя теор длина очереди

-20197661.288941357

In [ ]:
L = (p * n * math.pow(p, n)) / (math.factorial(n) * (1 - p))
for i in range(n):
    L += (math.pow(p * n, i) / math.factorial(i))
L # Среднее число заявок в СМО

20196491.257757593

In [ ]:
L = (p * n * math.pow(p, n)) / (math.factorial(n) * (1 - p))
for i in range(n):
    L += (math.pow(p * n, i) / math.factorial(i))
W = L / lam
W # Среднее время пребывания в системе

2735169.4552759468

In [ ]:
L_q = (p * n * math.pow(p, n)) / (math.factorial(n) * (1 - p) * (1 - math.pow(p, n+1)))
for i in range(n):
    L_q += (math.pow(p * n, i) / math.factorial(i))
L_q = p * n - L_q
W_q = L_q / lam
W_q # среднее время пребывания в очереди

-2735327.9102033256

# Система массового обслуживания (M|M|n|m)

In [ ]:
class Device:
    def __init__(self, device_id):
        self.device_id = device_id
        self.is_busy = False
        self.finish_time = None
        self.cur_request = None
        self.processing_time = None
        self.processed = 0
        self.was_busy = 0


    def finish(self, time):
      global requests
      if not self.is_busy: return
      print(f"DEVICE {self.device_id} HAS UNFINISHED JOB!")
      self.was_busy += time - requests[self.cur_request]["ts"]


    def new_request(self, request, write=True, override=False):
        global counter
        global actions
        global requests
        global state
        global left_time
        global next_query_deadline
        global next_next
        global min_finish

        self.is_busy = True
        self.processed += 1

        time = next_query_deadline if not override else override

        # Записать время и событие принятия заявки
        print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}')

        processing_time = np.random.exponential(1/mu)
        self.processing_time = processing_time
        if write:
          counter += 1
          actions.append([
            counter, #number
            time, # time recieved
            1, # type
            state, # state
            processing_time if processing_time < min_finish-time else min_finish-time, #min time left of work
            next_next, # next query in
            request, # request num
            self.device_id # device number
          ])

        self.finish_time = time + processing_time
        self.cur_request = request

        requests[request].update({
          "tq": time - requests[request]["tz"], # queue time
          "ts": time,
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
          "tk": self.device_id # who worked on request
        })


    def process_request(self):
        global counter
        global actions
        global devices
        global state
        global left_time
        global min_finish
        global next_query_deadline
        global min_device
        global devices
        global queue


        min_finish = np.inf
        min_device = None
        free_device = None

        for device in devices:
          if device.is_busy and device.finish_time < min_finish and device.device_id != self.device_id:
            min_finish = device.finish_time
            min_device = device
          elif not device.is_busy and free_device is None and device.device_id != self.device_id: free_device = device

        # Записать время и событие завершения обработки
        counter += 1
        state -= 1
        self.was_busy += self.processing_time

        print(f'{self.finish_time}: Устройство {self.device_id} обработало заявку {self.cur_request}, {counter}. Очередь: {len(queue)}')

        requests[self.cur_request].update({
          "tw": self.processing_time, # for how long worked
          "te": self.finish_time, # when finished working
        })

        # requests[self.cur_request].update({
        # })

        # есть ли новые заявки в очереди, добавить новый реквест
        if len(queue) != 0 and counter <= 100:
            self.processed += 1
            actions.append([
              counter, #number
              self.finish_time, # time recieved
              3, # type
              state, # state
              self.processing_time if self.processing_time < min_finish-self.finish_time else min_finish-self.finish_time, #min time left of work
              next_query_deadline - self.finish_time, # next query in
              self.cur_request, # request num
              self.device_id # device number
            ])

            request = queue.pop(0)
            time = self.finish_time

            # Записать время и событие принятия заявки
            print(f'{time}: Устройство {self.device_id} приняло заявку {request}, {counter}. Очередь: {len(queue)}')

            processing_time = np.random.exponential(1/mu)
            self.processing_time = processing_time

            self.finish_time = time + processing_time
            self.cur_request = request

            requests[request].update({
              "tq": time - requests[request]["tz"], # queue time
              "ts": time,
              "tw": self.processing_time, # for how long worked
              "te": self.finish_time, # when finished working
              "tk": self.device_id # who worked on request
            })
        elif counter <= 100:
          actions.append([
            counter, #number
            self.finish_time, # time recieved
            3, # type
            state, # state
            -1 if min_device is None else min_finish-self.finish_time,
            next_query_deadline - self.finish_time, # next query in
            self.cur_request, # request num
            self.device_id # device number
          ])

          self.finish_time = None
          self.cur_request = None
          self.is_busy = False

In [ ]:
counter = 0 # actions
request = 0 # queries
state = 0 # current state of query system
next_query_deadline = np.random.exponential(1/lam) # time when the new query appears
next_next =  np.random.exponential(1/lam)

queue = [] # queries queue # .pop(0)
requests = {} # log of requests
actions = [] # log of actions
zl = []

devices = [Device(i+1) for i in range(n)]
# watcher
while counter < 100:
  min_finish = np.inf
  min_device = None
  free_device = None

  # get device that will end sooner than others
  for device in devices:
    if device.is_busy and device.finish_time < min_finish:
      min_finish = device.finish_time
      min_device = device
    elif not device.is_busy and free_device is None: free_device = device

  # is there any query that fires sooner?
  if next_query_deadline < min_finish:
    request += 1
    if len(zl): zl.append(zl[-1]+1)
    else: zl.append(1)

    if free_device is None:
      # all devices are busy. Add to queue
      counter += 1

      if len(queue) >= m:
        print(f'{next_query_deadline}: ОТКАЗ. Новая заявка {request} получила отказ, {counter}. Очередь до события: {len(queue)}')
        state += 1
        actions.append([
            counter, #number
            next_query_deadline, # time recieved
            2, # type
            state, # state
            -1 if min_device is None else min_finish-next_query_deadline, #min time left of work
            next_next, # next query in
            request, # request num
            -1 # device number
        ])

        requests[request] = {
            "j": request,
            "tz": next_query_deadline, # appeared
            "q": -1, # queue pos
            "tq": 0, # queue time
            "ts": -1, # start of working on request
            "tw": 0, # for how long worked
            "te": next_query_deadline, # when finished working
            "tk": -2 # who worked on request
        }
      else:
        state += 1
        print(f'{next_query_deadline}: Новая заявка {request} встала в очередь, {counter}. Очередь до события: {len(queue)}')
        actions.append([
              counter, #number
              next_query_deadline, # time recieved
              1, # type
              state, # state
              -1 if min_device is None else min_finish-next_query_deadline, #min time left of work
              next_next, # next query in
              request, # request num
              -1 # device number
          ])
        queue.append(request)

        requests[request] = {
            "j": request,
            "tz": next_query_deadline, # appeared
            "q": len(queue), # queue pos
            "tq": -1, # queue time
            "ts": -1, # start of working on request
            "tw": -1, # for how long worked
            "te": -1, # when finished working
            "tk": -1 # who worked on request
        }
    else:
      # At least one device is free
      state += 1
      requests[request] = {
          "j": request,
          "tz": next_query_deadline, # appeared
          "q": 0, # queue pos
          "tq": 0, # queue time
          "ts": next_query_deadline, # start of working on request
          "tw": 0, # for how long worked
          "te": -1, # when finished working
          "tk": free_device.device_id # who worked on request
      }
      free_device.new_request(request)
    next_query_deadline +=  next_next
    next_next = np.random.exponential(1/lam)
  else:
    # Device processes query sooner
    zl.append(zl[-1]-1)
    min_device.process_request()


0.3095188720299117: Устройство 1 приняло заявку 1, 0
0.41305259732790456: Устройство 2 приняло заявку 2, 1
0.4284388250427368: Устройство 1 обработало заявку 1, 3. Очередь: 0
0.5405086941796834: Устройство 1 приняло заявку 3, 3
0.565208746722603: Устройство 3 приняло заявку 4, 4
0.57463423269698: Устройство 2 обработало заявку 2, 6. Очередь: 0
0.7565484754998242: Устройство 2 приняло заявку 5, 6
0.9179296966284253: Устройство 4 приняло заявку 6, 7
0.9464692480984014: Устройство 5 приняло заявку 7, 8
0.9661108241223663: Устройство 2 обработало заявку 5, 10. Очередь: 0
1.007272871501287: Устройство 2 приняло заявку 8, 10
1.0890806517035394: Устройство 6 приняло заявку 9, 11
1.1098883929091852: Устройство 4 обработало заявку 6, 13. Очередь: 0
1.41106122495809: Устройство 4 приняло заявку 10, 13
1.5519736541843625: Устройство 7 приняло заявку 11, 14
1.5759406219107135: Новая заявка 12 встала в очередь, 16. Очередь до события: 0
1.6100256583677413: Устройство 2 обработало заявку 8, 17. Очер

In [ ]:
for device in devices:
  device.finish(actions[-1][1])

DEVICE 1 HAS UNFINISHED JOB!
DEVICE 2 HAS UNFINISHED JOB!
DEVICE 3 HAS UNFINISHED JOB!
DEVICE 4 HAS UNFINISHED JOB!
DEVICE 5 HAS UNFINISHED JOB!
DEVICE 6 HAS UNFINISHED JOB!


In [ ]:
with open('actions3.pickle', 'wb') as handle:
    pickle.dump(actions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('requests3.pickle', 'wb') as handle:
    pickle.dump(requests, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('devices3.pickle', 'wb') as handle:
    pickle.dump(devices, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Вывод 1

In [ ]:
adf = pd.DataFrame(actions, columns=['l', 't_соб', 'Type(l)', 'C(l)', 't_остмин', "t_ожз", "j(l)", "k(l)"])
adf['t_соб'] = adf['t_соб'].round(5)
adf['t_остмин'] = adf['t_остмин'].round(5)
adf['t_ожз'] = adf['t_ожз'].round(5)

adf['k(l)'] = adf['k(l)'].astype(int)

adf[:100]

,l,t_соб,Type(l),C(l),t_остмин,t_ожз,j(l),k(l)
0,1,0.30952,1,1,0.11892,0.10353,1,1
1,2,0.41305,1,2,0.01539,0.12746,2,2
2,3,0.42844,3,1,0.14620,0.11207,1,1
3,4,0.54051,1,2,0.03413,0.02470,3,1
4,5,0.56521,1,3,0.00943,0.19134,4,3
...,...,...,...,...,...,...,...,...
95,96,6.95991,3,4,0.08800,0.20069,50,7
96,97,7.04790,3,3,0.22419,0.11269,46,4
97,98,7.16060,1,4,0.11150,0.02790,51,2
98,99,7.18850,1,5,0.08360,0.01073,52,3


In [ ]:
df1, df2, df3, df4 = np.array_split(adf, 4)
tables = [df1, df2, df3, df4]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-96-7c7b5d23a51f>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

 1 & 0.30952 &        1 &     1 &   0.11892 & 0.10353 &     1 &     1 \\ \hline
 2 & 0.41305 &        1 &     2 &   0.01539 & 0.12746 &     2 &     2 \\ \hline
 3 & 0.42844 &        3 &     1 &   0.14620 & 0.11207 &     1 &     1 \\ \hline
 4 & 0.54051 &        1 &     2 &   0.03413 & 0.02470 &     3 &     1 \\ \hline
 5 & 0.56521 &        1 &     3 &   0.00943 & 0.19134 &     4 &     3 \\ \hline
 6 & 0.57463 &        3 &     2 &   1.23474 & 0.18191 &     2 &     2 \\ \hline
 7 & 0.75655 &        1 &     3 &   0.20956 & 0.16138 &     5 &     2 \\ \hline
 8 & 0.91793 &        1 &     4 &   0.04818 & 0.02854 &     6 &     4 \\ \hline
 9 & 0.94647 &        1 &     5 &   0.01964 & 0.06080 &     7 &     5 \\ \hline
10 & 0.96611 &        3 &     4 &   0.14378 & 0.04116 &     5 &     2 \\ \hline
11 & 1.00727 &        1 &     5 &   0.10262 & 0.08181 &     8 &     2 \\ \

In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

26 & 2.01872 &        1 &     8 &   0.01613 & 0.73342 &    17 &    -1 \\ \hline
27 & 2.03485 &        3 &     7 &   0.01558 & 0.71729 &    16 &     7 \\ \hline
28 & 2.05043 &        3 &     6 &   0.04969 & 0.70171 &    14 &     3 \\ \hline
29 & 2.10012 &        3 &     5 &   0.03381 & 0.65202 &    15 &     5 \\ \hline
30 & 2.13393 &        3 &     4 &   0.00581 & 0.61821 &    17 &     7 \\ \hline
31 & 2.13974 &        3 &     3 &   0.27462 & 0.61240 &     3 &     1 \\ \hline
32 & 2.41436 &        3 &     2 &   0.09859 & 0.33778 &    10 &     4 \\ \hline
33 & 2.51295 &        3 &     1 &   0.66978 & 0.23919 &     9 &     6 \\ \hline
34 & 2.75214 &        1 &     2 &   0.43059 & 0.26521 &    18 &     1 \\ \hline
35 & 3.01735 &        1 &     3 &   0.16538 & 0.01574 &    19 &     3 \\ \hline
36 & 3.03310 &        1 &     4 &   0.07234 & 0.01722 &    20 &     4 \\ \

In [ ]:
print(latex_tables[2])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

51 & 3.81667 &        3 &     7 &   0.04453 & 0.02060 &    23 &     2 \\ \hline
52 & 3.83228 &        3 &     6 &   0.02892 & 0.00498 &    29 &     2 \\ \hline
53 & 3.83726 &        1 &     7 &   0.02394 & 0.07573 &    30 &     2 \\ \hline
54 & 3.86120 &        3 &     6 &   0.21771 & 0.05180 &    28 &     7 \\ \hline
55 & 3.91300 &        1 &     7 &   0.16591 & 0.32387 &    31 &     7 \\ \hline
56 & 4.07891 &        3 &     6 &   0.10532 & 0.15797 &    19 &     3 \\ \hline
57 & 4.18423 &        3 &     5 &   0.02519 & 0.05264 &    30 &     2 \\ \hline
58 & 4.20942 &        3 &     4 &   0.09375 & 0.02746 &    24 &     4 \\ \hline
59 & 4.23687 &        1 &     5 &   0.06629 & 0.23402 &    32 &     2 \\ \hline
60 & 4.30316 &        3 &     4 &   0.12274 & 0.16773 &    27 &     5 \\ \hline
61 & 4.42590 &        3 &     3 &   0.00636 & 0.04499 &    32 &     2 \\ \

In [ ]:
print(latex_tables[3])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
  l &   t\_соб &  Type(l) &  C(l) &  t\_остмин &   t\_ожз &  j(l) &  k(l) \\ \hline

 76 & 5.62794 &        1 &     6 &   0.11096 & 0.00569 &    41 &     6 \\ \hline
 77 & 5.63364 &        1 &     7 &   0.10526 & 0.22938 &    42 &     7 \\ \hline
 78 & 5.73890 &        3 &     6 &   0.01636 & 0.12412 &    40 &     5 \\ \hline
 79 & 5.75526 &        3 &     5 &   0.02022 & 0.10776 &    41 &     6 \\ \hline
 80 & 5.77548 &        3 &     4 &   0.03820 & 0.08754 &    42 &     7 \\ \hline
 81 & 5.81368 &        3 &     3 &   0.08106 & 0.04934 &    39 &     4 \\ \hline
 82 & 5.86302 &        1 &     4 &   0.03172 & 0.13011 &    43 &     4 \\ \hline
 83 & 5.89474 &        3 &     3 &   0.09444 & 0.09839 &    38 &     3 \\ \hline
 84 & 5.98917 &        3 &     2 &   0.02329 & 0.00396 &    37 &     1 \\ \hline
 85 & 5.99313 &        1 &     3 &   0.01933 & 0.26014 &    44 &     1 \\ \hline
 86 & 6.01246 &        3 &     2 &   0.58934 & 0.24081 &    43 

## Вывод 2

In [ ]:
rdf = pd.DataFrame.from_dict(requests).transpose()
rdf['j'] = rdf['j'].astype(int)

rdf['tw'] = rdf['tw'].round(5)
rdf['te'] = rdf['te'].round(5)
rdf['tq'] = rdf['tq'].round(5)
rdf['ts'] = rdf['ts'].round(5)
rdf['tz'] = rdf['tz'].round(5)

rdf['q'] = rdf['q'].astype(int)
rdf['tk'] = rdf['tk'].astype(int)
rdf

,j,tz,q,tq,ts,tw,te,tk
1,1,0.30952,0,0.00000,0.30952,0.11892,0.42844,1
2,2,0.41305,0,0.00000,0.41305,0.16158,0.57463,2
3,3,0.54051,0,0.00000,0.54051,1.59923,2.13974,1
4,4,0.56521,0,0.00000,0.56521,1.24416,1.80937,3
5,5,0.75655,0,0.00000,0.75655,0.20956,0.96611,2
6,6,0.91793,0,0.00000,0.91793,0.19196,1.10989,4
7,7,0.94647,0,0.00000,0.94647,0.92214,1.86861,5
8,8,1.00727,0,0.00000,1.00727,0.60275,1.61003,2
9,9,1.08908,0,0.00000,1.08908,1.42387,2.51295,6
10,10,1.41106,0,0.00000,1.41106,1.00330,2.41436,4


In [ ]:
df1, df2, df3 = rdf[:25], rdf[25:50], rdf[50:]
tables = [df1, df2, df3]

In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')

    return latex

# Используем функцию
latex_tables = [latex_with_lines(df) for df in tables]

<ipython-input-103-7c7b5d23a51f>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_tables[0])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &      tq &      ts &      tw &      te &  tk \\ \hline

 1 & 0.30952 &  0 & 0.00000 & 0.30952 & 0.11892 & 0.42844 &   1 \\ \hline
 2 & 0.41305 &  0 & 0.00000 & 0.41305 & 0.16158 & 0.57463 &   2 \\ \hline
 3 & 0.54051 &  0 & 0.00000 & 0.54051 & 1.59923 & 2.13974 &   1 \\ \hline
 4 & 0.56521 &  0 & 0.00000 & 0.56521 & 1.24416 & 1.80937 &   3 \\ \hline
 5 & 0.75655 &  0 & 0.00000 & 0.75655 & 0.20956 & 0.96611 &   2 \\ \hline
 6 & 0.91793 &  0 & 0.00000 & 0.91793 & 0.19196 & 1.10989 &   4 \\ \hline
 7 & 0.94647 &  0 & 0.00000 & 0.94647 & 0.92214 & 1.86861 &   5 \\ \hline
 8 & 1.00727 &  0 & 0.00000 & 1.00727 & 0.60275 & 1.61003 &   2 \\ \hline
 9 & 1.08908 &  0 & 0.00000 & 1.08908 & 1.42387 & 2.51295 &   6 \\ \hline
10 & 1.41106 &  0 & 0.00000 & 1.41106 & 1.00330 & 2.41436 &   4 \\ \hline
11 & 1.55197 &  0 & 0.00000 & 1.55197 & 0.17559 & 1.72757 &   7 \\ \hline
12 & 1.57594 &  1 & 0.03409 & 1.61003 & 0.12965 & 1.73968 &   2 \\ \h

In [ ]:
print(latex_tables[1])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &      tq &      ts &      tw &       te &  tk \\ \hline

26 & 3.66799 &  0 & 0.00000 & 3.66799 & 0.01837 &  3.68636 &   7 \\ \hline
27 & 3.70884 &  0 & 0.00000 & 3.70884 & 0.59432 &  4.30316 &   5 \\ \hline
28 & 3.73997 &  0 & 0.00000 & 3.73997 & 0.12123 &  3.86120 &   7 \\ \hline
29 & 3.80080 &  1 & 0.01586 & 3.81667 & 0.01562 &  3.83228 &   2 \\ \hline
30 & 3.83726 &  0 & 0.00000 & 3.83726 & 0.34697 &  4.18423 &   2 \\ \hline
31 & 3.91300 &  0 & 0.00000 & 3.91300 & 1.00903 &  4.92203 &   7 \\ \hline
32 & 4.23687 &  0 & 0.00000 & 4.23687 & 0.18903 &  4.42590 &   2 \\ \hline
33 & 4.47089 &  0 & 0.00000 & 4.47089 & 2.13091 &  6.60180 &   2 \\ \hline
34 & 4.48791 &  0 & 0.00000 & 4.48791 & 0.26889 &  4.75680 &   3 \\ \hline
35 & 4.54836 &  0 & 0.00000 & 4.54836 & 0.38986 &  4.93823 &   4 \\ \hline
36 & 4.56098 &  0 & 0.00000 & 4.56098 & 0.43609 &  4.99708 &   5 \\ \hline
37 & 5.21321 &  0 & 0.00000 & 5.21321 & 0.77596 &  5.9891

In [ ]:
print(latex_tables[2])

\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
 j &      tz &  q &  tq &      ts &      tw &      te &  tk \\ \hline

51 & 7.16060 &  0 & 0.0 & 7.16060 & 0.75118 & 7.91178 &   2 \\ \hline
52 & 7.18850 &  0 & 0.0 & 7.18850 & 0.31085 & 7.49935 &   3 \\ \hline
53 & 7.19923 &  0 & 0.0 & 7.19923 & 0.09092 & 7.29015 &   4 \\ \hline

\end{tabular}



## Вывод 3

In [ ]:
final_t = adf["t_соб"][99]
final_t

7.19923

In [ ]:
ddf = pd.DataFrame([[device.device_id, device.processed, device.was_busy, final_t-device.was_busy, (final_t-device.was_busy)/final_t] for device in devices],
                   columns=['k', 'N(k)', 't_зан', 't_np', "Delta"])
ddf.loc[len(ddf.index)] = [None, sum(ddf["N(k)"]), sum(ddf["N(k)"])/n,  sum(ddf["t_np"])/n,  sum(ddf["Delta"])/n]

ddf['k'] = ddf['k'].astype("Int64")
ddf['N(k)'] = ddf['N(k)'].astype(int)
ddf['t_зан'] = ddf['t_зан'].round(5)
ddf['t_np'] = ddf['t_np'].round(5)
ddf['Delta'] = ddf['Delta'].round(5)

ddf

,k,N(k),t_зан,t_np,Delta
0,1,5,6.00618,1.19305,0.16572
1,2,11,5.63264,1.56659,0.21760
2,3,8,3.90386,3.29537,0.45774
3,4,10,3.85500,3.34423,0.46453
4,5,7,3.82913,3.37010,0.46812
5,6,4,3.16354,4.03569,0.56057
6,7,8,1.98999,5.20924,0.72358
7,<NA>,53,7.57143,3.14490,0.43684


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')
    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
latex_table = latex_with_lines(ddf)

<ipython-input-109-24eb57417011>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_table)

\begin{tabular}{|c|c|c|c|c|}
\hline
   k &  N(k) &   t\_зан &    t\_np &   Delta \\ \hline

   1 &     5 & 6.00618 & 1.19305 & 0.16572 \\ \hline
   2 &    11 & 5.63264 & 1.56659 & 0.21760 \\ \hline
   3 &     8 & 3.90386 & 3.29537 & 0.45774 \\ \hline
   4 &    10 & 3.85500 & 3.34423 & 0.46453 \\ \hline
   5 &     7 & 3.82913 & 3.37010 & 0.46812 \\ \hline
   6 &     4 & 3.16354 & 4.03569 & 0.56057 \\ \hline
   7 &     8 & 1.98999 & 5.20924 & 0.72358 \\ \hline
  &    53 & 7.57143 & 3.14490 & 0.43684 \\ \hline

\end{tabular}



## Вывод 4

In [ ]:
adf['C(l)'].value_counts().sort_index().index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [ ]:
R = adf['C(l)'].value_counts().sort_index().values
if adf['C(l)'].value_counts().sort_index().index[0] != 0:
  R = np.insert(R, 0, 0)
R

array([ 0,  4, 10, 15, 20, 20, 17, 11,  3])

In [ ]:
sum([r/100 for r in R])

1.0

In [ ]:
s = final_t

T = [0]*(len(R))
D = [0]*len(R)
cur_state = 0
last_time = 0

for e in actions:
  _st = e[3]
  if _st != cur_state:
    # print(_st, e[1])
    T[cur_state] += e[1] - last_time
    last_time, cur_state = e[1], e[3]

for i in range(len(T)):
  D[i] = T[i]/s
print(T)
print(s, D, sum(D))

[0.3095188720299117, 0.609889193268722, 1.0441636055031878, 1.027821038261366, 0.9749508473206301, 1.362136520906271, 1.2070399415413602, 0.5976309038786043, 0.06607473978554523]
7.19923 [0.04299333012418157, 0.08471589229247045, 0.1450382340199143, 0.14276819024553541, 0.13542432278460753, 0.18920586241949083, 0.16766236688386954, 0.08301317000270922, 0.009178028731620637] 0.9999993975043994


In [ ]:
states = np.array(adf['C(l)'].value_counts().sort_index().index)
if states[0] != 0:
  states = np.insert(states, 0, 0)
states

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
import math

def calculate_P0(lam, mu, n, m):
  rho = lam/mu
  nu = rho/n
  return 3.1837621363823E-7 #
  return np.sum([(rho**k)/(factorial(k)) for k in range(n)] + [((rho**n)/factorial(n)) * np.sum([nu**k for k in range(m+1)])])**-1

def calculate_Pi(lam, mu, n, m, i):
    rho = lam/mu
    return (rho**i/factorial(i)) * calculate_P0(lam ,mu, n, m)


p0 = calculate_P0(lam, mu, n, m)
Pi = calculate_Pi(lam, mu, n, m, 19)
print(p0, Pi)

3.1837621363823e-07 2.316389916041767e-08


In [ ]:
n+m

19

In [ ]:
np.sum([calculate_Pi(lam, mu, n, m, i) for i in range(20)])

0.0003182481615306811

In [ ]:
sdf = pd.DataFrame({
    'Состояние': states,
    "r_i": [calculate_Pi(lam, mu, n, m, i) for i in range(len(R))],
    'R': R
})
sdf['v'] = sdf["R"]/100
sdf['T'] = T
sdf['Delta'] = D

sdf.loc[len(sdf.index)] = [None, sum(sdf["r_i"]), sum(sdf["R"]),  sum(sdf["v"]), sum(sdf["T"]), sum(sdf["Delta"])]

sdf['Состояние'] = sdf['Состояние'].astype("Int64")
sdf['r_i'] = sdf['r_i'].round(5)
sdf['R'] = sdf['R'].astype(int)
sdf['T'] = sdf['T'].round(5)
sdf['Delta'] = sdf['Delta'].round(5)
sdf

,Состояние,r_i,R,v,T,Delta
0,0,0.00000,0,0.00,0.30952,0.04299
1,1,0.00000,4,0.04,0.60989,0.08472
2,2,0.00001,10,0.10,1.04416,0.14504
3,3,0.00002,15,0.15,1.02782,0.14277
4,4,0.00003,20,0.20,0.97495,0.13542
5,5,0.00004,20,0.20,1.36214,0.18921
6,6,0.00005,17,0.17,1.20704,0.16766
7,7,0.00005,11,0.11,0.59763,0.08301
8,8,0.00004,3,0.03,0.06607,0.00918
9,<NA>,0.00024,100,1.00,7.19923,1.00000


In [ ]:
def latex_with_lines(df):
    column_format = '|c|c|c|c|c|'
    latex = df.to_latex(index=False, column_format=column_format)

    # Добавляем горизонтальные линии
    latex = latex.replace('\\toprule', '\\hline')

    latex = latex.replace('\\midrule', '')
    latex = latex.replace('\\bottomrule', '')

    latex = latex.replace('\\\\\n', '\\\\ \\hline\n')
    latex = latex.replace('<NA>', ' ')

    return latex

# Используем функцию
latex_table = latex_with_lines(sdf)

<ipython-input-120-edcf29cf9fc3>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False, column_format=column_format)


In [ ]:
print(latex_table)

\begin{tabular}{|c|c|c|c|c|}
\hline
 Состояние &     r\_i &   R &    v &       T &   Delta \\ \hline

         0 & 0.00000 &   0 & 0.00 & 0.30952 & 0.04299 \\ \hline
         1 & 0.00000 &   4 & 0.04 & 0.60989 & 0.08472 \\ \hline
         2 & 0.00001 &  10 & 0.10 & 1.04416 & 0.14504 \\ \hline
         3 & 0.00002 &  15 & 0.15 & 1.02782 & 0.14277 \\ \hline
         4 & 0.00003 &  20 & 0.20 & 0.97495 & 0.13542 \\ \hline
         5 & 0.00004 &  20 & 0.20 & 1.36214 & 0.18921 \\ \hline
         6 & 0.00005 &  17 & 0.17 & 1.20704 & 0.16766 \\ \hline
         7 & 0.00005 &  11 & 0.11 & 0.59763 & 0.08301 \\ \hline
         8 & 0.00004 &   3 & 0.03 & 0.06607 & 0.00918 \\ \hline
        & 0.00024 & 100 & 1.00 & 7.19923 & 1.00000 \\ \hline

\end{tabular}



## Вывод 5

In [ ]:
print(len(rdf)) #число заявок

53


In [ ]:
rdf[(rdf["te"] > adf["t_соб"][99]) | (rdf["tk"] == -2) | (rdf["tk"] == -1)] # Эти заявки не завершились на последнем событии или стоят в очереди

,j,tz,q,tq,ts,tw,te,tk
44,44,5.99313,0,0.0,5.99313,1.27897,7.27210,1
47,47,6.32789,0,0.0,6.32789,1.20731,7.53520,5
49,49,6.52802,0,0.0,6.52802,3.65846,10.18648,6
51,51,7.16060,0,0.0,7.16060,0.75118,7.91178,2
52,52,7.18850,0,0.0,7.18850,0.31085,7.49935,3
53,53,7.19923,0,0.0,7.19923,0.09092,7.29015,4


In [ ]:
obsl = len(rdf[(rdf["tk"] != -1) & (rdf["te"] <= adf["t_соб"][99]) & (rdf["tk"] != -2)]) # Обработанные заявки
obsl

47

In [ ]:
adf["C(l)"].sum()/100 # среднее число заявок

4.52

In [ ]:
rdf["tq"][(rdf["tq"] >= 0) & (rdf["tk"] != -2)].sum()/obsl # среднее пребывание в очереди

0.0014059574468085105

In [ ]:
np.round(0.07469825301816856, 5)

0.0747

In [ ]:
(rdf["te"][rdf["tq"] >= 0] - rdf["tz"][rdf["tq"] >= 0]).sum()/obsl # среднее пребывание в СМО

0.7009812765957445

In [ ]:
np.round(1.5607197826808805, 5)

1.56072

In [ ]:
p = lam/(mu)
p

6.907390084190833